In [1]:
import matplotlib.pyplot as plt
import glob
import numpy as np

In [ ]:
data = 'Data/Predictions/Virgo5_R12.5_D12.75_f2.000_p76FAP_Data.txt'

raw=[]
for txt in glob.glob(data + '*.txt'):
    with open(txt, 'r') as f:
        raw.append(f.read())